In [1]:
import itertools
from parlay_system import *
pd.set_option('display.max_rows', 500)


In [2]:
# Dec 15 games
# flyers vs jets
phi_m15 = MoneyLine(event="phi_m15", bet_amount=100, odds=165)
phi_p15 = MoneyLine(event="phi_p15", bet_amount=100, odds=-200)

# kings vs red wings
kings_m15 = MoneyLine(event="kings_m15", bet_amount=100, odds=210)
kings_p15 = MoneyLine(event="kings_p15", bet_amount=100, odds=-265)

# wild vs blackhawks
wild_m15 = MoneyLine(event="wild_m15", bet_amount=100, odds=225)
wild_p15 = MoneyLine(event="wild_p15", bet_amount=100, odds=-278)

# canucks vs golden knights
canucks_m15 = MoneyLine(event="canucks_m15", bet_amount=100, odds=-182)
canucks_p15 = MoneyLine(event="canucks_p15", bet_amount=100, odds=150)

binaries = [
    [phi_m15, phi_p15],
    [kings_m15, kings_p15],
    [wild_m15, wild_p15],
    [canucks_m15, canucks_p15]
]

results = [
    [0, 1], # [phi_m15, phi_p15],
    [0, 1], # [kings_m15, kings_p15],
    [0, 1], # [wild_m15, wild_p15],
    [1, 0]  # [canucks_m15, canucks_p15]
]


def create_dicts(in_binaries, in_results):
    ml_lookup = {}
    ml_win_lookup = {}
    id = 0
    for bi in range(len(in_binaries)):
        binary = in_binaries[bi]
        for ml_i in range(len(binary)):
            ml = binary[ml_i]
            ml.set_index(id) 
            is_winner = in_results[bi][ml_i]
            ml_lookup[id] = ml.event
            ml_win_lookup[id] = is_winner
            id += 1
    
    return ml_lookup, ml_win_lookup

ml_lookup, ml_outcome_lookup = create_dicts(binaries, results)



In [3]:
#ml_lookup

In [4]:
#ml_outcome_lookup

In [5]:
def findsubsets(s, n):
    return list(itertools.combinations(s, n))

TOURNEY_SIZE = len(binaries)
SELECT_NUM = 3
s = set()
n = SELECT_NUM

for i in range(TOURNEY_SIZE):
    s.add(i)

all_subsets = findsubsets(s, n)

In [6]:
len(all_subsets)

4

In [7]:
csv = []
df = pd.DataFrame()

for i in range(len(all_subsets)):
    current_subset = all_subsets[i]
    parlay_binaries = []
    for i in range(len(current_subset)):
        index = current_subset[i]
        parlay_binaries.append(binaries[index])

    ps = ParlaySystem(binaries=parlay_binaries,
                    target_profit=0.50,
                    bounds=(0.01, 30),
                    binary_index_arr=list(current_subset),
                    binary_results_arr=results,
                    index_to_ml=ml_lookup,
                    index_to_outcome=ml_outcome_lookup
                    )
    csv_res, raw_df = ps.slsqp_solver()
    df = pd.concat([df, raw_df])
    csv.append(csv_res)

     fun: -0.25504134646299603
     jac: array([-2.46234375, -0.5212545 , -0.6078125 ,  0.2546313 , -1.0140625 ,
        0.08466727,  0.03567217,  0.5238479 ])
 message: 'Positive directional derivative for linesearch'
    nfev: 50
     nit: 8
    njev: 4
  status: 8
 success: False
       x: array([0.04333527, 0.10358048, 0.09753404, 0.09174736, 0.07655898,
       0.1146779 , 0.11956361, 0.01      ])
slsqp_solver: 
                         event    index[]     result  event_status   bet  \
5  phi_p15_kings_m15_wild_p15_  [1, 2, 5]  [1, 1, 1]          True  0.11   
0  phi_m15_kings_m15_wild_m15_  [0, 2, 4]  [0, 1, 0]         False  0.04   
1  phi_m15_kings_m15_wild_p15_  [0, 2, 5]  [0, 1, 1]         False  0.10   
2  phi_m15_kings_p15_wild_m15_  [0, 3, 4]  [0, 0, 0]         False  0.10   
4  phi_p15_kings_m15_wild_m15_  [1, 2, 4]  [1, 1, 0]         False  0.08   
6  phi_p15_kings_p15_wild_m15_  [1, 3, 4]  [1, 0, 0]         False  0.12   
3  phi_m15_kings_p15_wild_p15_  [0, 3, 5]  [0, 0

In [8]:
df.describe()

,bet,mult,payout,profit
count,32.000000,32.000000,32.000000,32.000000
mean,0.086875,10.060669,0.853972,0.157187
std,0.037797,6.323120,0.420722,0.420930
min,0.010000,2.809217,0.028100,-0.710000
25%,0.075000,5.478552,0.548025,-0.162500
50%,0.100000,8.280832,1.083500,0.395000
75%,0.110000,12.322809,1.184550,0.500000
max,0.140000,26.698750,1.237500,0.500000


In [9]:
negative_df = df.loc[df.profit < 0]
negative_df = negative_df.sort_values(by=['event_status', 'profit'], ascending=[False, True])
negative_df.describe()

,bet,mult,payout,profit
count,10.000000,10.000000,10.000000,10.000000
mean,0.059000,4.322000,0.307220,-0.397000
std,0.042544,1.161157,0.244919,0.237863
min,0.010000,2.809217,0.028100,-0.710000
25%,0.010000,3.170464,0.031700,-0.637500
50%,0.085000,4.822489,0.423950,-0.270000
75%,0.090000,5.148551,0.474925,-0.215000
max,0.100000,5.655495,0.590400,-0.150000


In [10]:
positive_df = df.loc[df.profit >= 0]
positive_df = positive_df.sort_values(by=['event_status', 'profit'], ascending=[False, False])
positive_df.describe()

,bet,mult,payout,profit
count,22.000000,22.000000,22.000000,22.000000
mean,0.099545,12.669155,1.102495,0.409091
std,0.028196,5.969315,0.168911,0.156780
min,0.040000,6.322662,0.725100,0.070000
25%,0.082500,7.917202,1.046750,0.342500
50%,0.100000,11.408019,1.172500,0.500000
75%,0.120000,14.670536,1.220700,0.500000
max,0.140000,26.698750,1.237500,0.500000


In [11]:
negative_df.head(200)

,event,index[],result,event_status,bet,mult,payout,profit
6,phi_p15_wild_p15_canucks_m15_,"[1, 5, 6]","[1, 1, 1]",True,0.01,3.160210,0.0316,-0.69
6,phi_p15_kings_p15_canucks_m15_,"[1, 3, 6]","[1, 0, 1]",False,0.01,3.201223,0.0320,-0.71
6,kings_p15_wild_p15_canucks_m15_,"[3, 5, 6]","[0, 1, 1]",False,0.01,2.901828,0.0290,-0.64
7,phi_p15_kings_p15_wild_p15_,"[1, 3, 5]","[1, 0, 1]",False,0.01,2.809217,0.0281,-0.63
7,kings_p15_wild_p15_canucks_p15_,"[3, 5, 7]","[0, 1, 0]",False,0.08,4.682028,0.3926,-0.28
7,phi_p15_kings_p15_canucks_p15_,"[1, 3, 7]","[1, 0, 0]",False,0.09,5.165094,0.4788,-0.26
7,phi_p15_wild_p15_canucks_p15_,"[1, 5, 7]","[1, 1, 0]",False,0.09,5.098921,0.4633,-0.26
3,phi_m15_kings_p15_wild_p15_,"[0, 3, 5]","[0, 0, 1]",False,0.09,4.962950,0.4553,-0.20
2,phi_m15_kings_p15_canucks_m15_,"[0, 3, 6]","[0, 0, 1]",False,0.10,5.655495,0.5904,-0.15
2,phi_m15_wild_p15_canucks_m15_,"[0, 5, 6]","[0, 1, 1]",False,0.10,5.583038,0.5711,-0.15


In [12]:
positive_df.head(200)

,event,index[],result,event_status,bet,mult,payout,profit
4,phi_p15_kings_m15_canucks_m15_,"[1, 2, 6]","[1, 1, 1]",True,0.13,7.204945,0.9429,0.21
2,kings_m15_wild_p15_canucks_m15_,"[2, 5, 6]","[1, 1, 1]",True,0.12,6.531101,0.7703,0.10
5,phi_p15_kings_m15_wild_p15_,"[1, 2, 5]","[1, 1, 1]",True,0.11,6.322662,0.7251,0.07
0,phi_m15_kings_m15_wild_m15_,"[0, 2, 4]","[0, 1, 0]",False,0.04,26.698750,1.1570,0.50
1,phi_m15_kings_m15_wild_p15_,"[0, 2, 5]","[0, 1, 1]",False,0.10,11.170036,1.1570,0.50
2,phi_m15_kings_p15_wild_m15_,"[0, 3, 4]","[0, 0, 0]",False,0.10,11.862500,1.1570,0.50
4,phi_p15_kings_m15_wild_m15_,"[1, 2, 4]","[1, 1, 0]",False,0.08,15.112500,1.1570,0.50
0,phi_m15_kings_m15_canucks_m15_,"[0, 2, 6]","[0, 1, 1]",False,0.10,12.728736,1.2375,0.50
1,phi_m15_kings_m15_canucks_p15_,"[0, 2, 7]","[0, 1, 0]",False,0.06,20.537500,1.2375,0.50
3,phi_m15_kings_p15_canucks_p15_,"[0, 3, 7]","[0, 0, 0]",False,0.14,9.125000,1.2375,0.50


In [13]:
positive_df.groupby('event_status').count()

,event,index[],result,bet,mult,payout,profit
event_status,,,,,,,
False,19,19,19,19,19,19,19
True,3,3,3,3,3,3,3


In [14]:
negative_df.groupby('event_status').count()

,event,index[],result,bet,mult,payout,profit
event_status,,,,,,,
False,9,9,9,9,9,9,9
True,1,1,1,1,1,1,1


In [15]:
#csv

In [16]:
def color_negative_red(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for negative
    strings, black otherwise.
    """
    if type(val) != bool:
        return 'color: black'
    
    color = 'red' if val == True else 'black'
    return 'color: %s' % color

hl = negative_df.reset_index(drop=True).style.applymap(color_negative_red)

In [17]:
hl

,event,index[],result,event_status,bet,mult,payout,profit
0,phi_p15_wild_p15_canucks_m15_,"['1', '5', '6']","[1, 1, 1]",True,0.01,3.16021,0.0316,-0.69
1,phi_p15_kings_p15_canucks_m15_,"['1', '3', '6']","[1, 0, 1]",False,0.01,3.20122,0.032,-0.71
2,kings_p15_wild_p15_canucks_m15_,"['3', '5', '6']","[0, 1, 1]",False,0.01,2.90183,0.029,-0.64
3,phi_p15_kings_p15_wild_p15_,"['1', '3', '5']","[1, 0, 1]",False,0.01,2.80922,0.0281,-0.63
4,kings_p15_wild_p15_canucks_p15_,"['3', '5', '7']","[0, 1, 0]",False,0.08,4.68203,0.3926,-0.28
5,phi_p15_kings_p15_canucks_p15_,"['1', '3', '7']","[1, 0, 0]",False,0.09,5.16509,0.4788,-0.26
6,phi_p15_wild_p15_canucks_p15_,"['1', '5', '7']","[1, 1, 0]",False,0.09,5.09892,0.4633,-0.26
7,phi_m15_kings_p15_wild_p15_,"['0', '3', '5']","[0, 0, 1]",False,0.09,4.96295,0.4553,-0.2
8,phi_m15_kings_p15_canucks_m15_,"['0', '3', '6']","[0, 0, 1]",False,0.1,5.65549,0.5904,-0.15
9,phi_m15_wild_p15_canucks_m15_,"['0', '5', '6']","[0, 1, 1]",False,0.1,5.58304,0.5711,-0.15
